In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from nltk.corpus import stopwords
from wordcloud import WordCloud ,STOPWORDS
from nltk.tokenize import word_tokenize 
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import re
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('BigBasket Products.csv')

In [3]:
df.drop('index',axis=1,inplace=True)

In [4]:
df.head()

,product,category,sub_category,brand,sale_price,market_price,type,rating,description
0,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
1,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
2,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
3,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
4,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...


Handling Duplicate values 

In [5]:
df.duplicated().sum()

354

In [6]:
df.drop_duplicates(inplace=True)

In [7]:
df.duplicated().sum()

0

Handling Missing Values

In [8]:
df.isnull().sum()

product            1
category           0
sub_category       0
brand              1
sale_price         0
market_price       0
type               0
rating          8463
description      113
dtype: int64

First Rating column will be handled 

In [9]:
df.groupby('category')['rating'].mean().round(2)

category
Baby Care                   4.02
Bakery, Cakes & Dairy       3.91
Beauty & Hygiene            3.93
Beverages                   4.08
Cleaning & Household        3.96
Eggs, Meat & Fish            NaN
Foodgrains, Oil & Masala    4.06
Fruits & Vegetables          NaN
Gourmet & World Food        3.98
Kitchen, Garden & Pets      3.74
Snacks & Branded Foods      3.98
Name: rating, dtype: float64

In [10]:
df.dropna(subset = ['product','brand','description'],axis=0,inplace=True)

After comparing with Label encoder, KNN imputation and mean values Let's go ahead with KNN imputation 

In [11]:
DF = df.copy()

In [12]:
le = LabelEncoder()

In [13]:
DF['category'] = le.fit_transform(df['category'])
DF['rating_le'] = DF['rating']

In [14]:
DF['rating_le'].isnull().sum()

8437

In [15]:
DF.head()

,product,category,sub_category,brand,sale_price,market_price,type,rating,description,rating_le
0,Garlic Oil - Vegetarian Capsule 500 mg,2,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...,4.1
1,Water Bottle - Orange,9,Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ...",2.3
2,"Brass Angle Deep - Plain, No.2",4,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m...",3.4
3,Cereal Flip Lid Container/Storage Jar - Assort...,4,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...,3.7
4,Creme Soft Soap - For Hands & Body,2,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...,4.4


In [16]:
impute = KNNImputer(n_neighbors=3)
DF[['category','rating_le','sale_price','market_price']] = impute.fit_transform(DF[['category','rating_le','sale_price','market_price']])

In [17]:
DF.drop('rating',axis =1,inplace=True)

In [18]:
DF.isnull().sum()

product         0
category        0
sub_category    0
brand           0
sale_price      0
market_price    0
type            0
description     0
rating_le       0
dtype: int64

In [19]:
DF.rename(columns={'rating_le':'rating'},inplace = True)

In [20]:
DF.head()

,product,category,sub_category,brand,sale_price,market_price,type,description,rating
0,Garlic Oil - Vegetarian Capsule 500 mg,2.0,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,This Product contains Garlic Oil that is known...,4.1
1,Water Bottle - Orange,9.0,Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,"Each product is microwave safe (without lid), ...",2.3
2,"Brass Angle Deep - Plain, No.2",4.0,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,"A perfect gift for all occasions, be it your m...",3.4
3,Cereal Flip Lid Container/Storage Jar - Assort...,4.0,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",Multipurpose container with an attractive desi...,3.7
4,Creme Soft Soap - For Hands & Body,2.0,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,Nivea Creme Soft Soap gives your skin the best...,4.4


In [21]:
DF.shape

(27087, 9)

In [22]:
DF['description']

0        This Product contains Garlic Oil that is known...
1        Each product is microwave safe (without lid), ...
2        A perfect gift for all occasions, be it your m...
3        Multipurpose container with an attractive desi...
4        Nivea Creme Soft Soap gives your skin the best...
                               ...                        
27550    Layerr brings you Wottagirl Classic fragrant b...
27551    Puramate rosemary is enough to transform a dis...
27552    We have taken the richness of Sweet Potatoes (...
27553    Tetley Green Tea with its refreshing pure, ori...
27554    The new mens fragrance from the United Dreams ...
Name: description, Length: 27087, dtype: object

Removing 
1. \R
2.\N
3. Punctuations and stopwords

In [23]:
def process_text(text):
    text = text.lower().replace('\n',' ').replace('\r','').strip()
    text = re.sub(' +', ' ', text)
    text = re.sub(r'[^\w\s]','',text)
    
    
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(text) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    filtered_sentence = [] 
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w) 
    
    text = " ".join(filtered_sentence)
    return text

In [24]:
DF['description_parsed'] = DF['description'].apply(process_text)

In [25]:
DF['description_parsed']

0        product contains garlic oil known help proper ...
1        product microwave safe without lid refrigerato...
2        perfect gift occasions mother sister inlaws bo...
3        multipurpose container attractive design made ...
4        nivea creme soft soap gives skin best care mus...
                               ...                        
27550    layerr brings wottagirl classic fragrant body ...
27551    puramate rosemary enough transform dish someth...
27552    taken richness sweet potatoes shakarkand given...
27553    tetley green tea refreshing pure original flav...
27554    new mens fragrance united dreams collection de...
Name: description_parsed, Length: 27087, dtype: object

In [26]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27087 entries, 0 to 27554
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   product             27087 non-null  object 
 1   category            27087 non-null  float64
 2   sub_category        27087 non-null  object 
 3   brand               27087 non-null  object 
 4   sale_price          27087 non-null  float64
 5   market_price        27087 non-null  float64
 6   type                27087 non-null  object 
 7   description         27087 non-null  object 
 8   rating              27087 non-null  float64
 9   description_parsed  27087 non-null  object 
dtypes: float64(4), object(6)
memory usage: 2.3+ MB


In [27]:
DF['product']

0                   Garlic Oil - Vegetarian Capsule 500 mg
1                                    Water Bottle - Orange
2                           Brass Angle Deep - Plain, No.2
3        Cereal Flip Lid Container/Storage Jar - Assort...
4                       Creme Soft Soap - For Hands & Body
                               ...                        
27550           Wottagirl! Perfume Spray - Heaven, Classic
27551                                             Rosemary
27552                         Peri-Peri Sweet Potato Chips
27553                            Green Tea - Pure Original
27554                       United Dreams Go Far Deodorant
Name: product, Length: 27087, dtype: object

In [28]:
DF['product_parsed'] = DF['product'].apply(process_text)

In [29]:
DF['product_parsed']

0                     garlic oil vegetarian capsule 500 mg
1                                      water bottle orange
2                               brass angle deep plain no2
3        cereal flip lid containerstorage jar assorted ...
4                               creme soft soap hands body
                               ...                        
27550               wottagirl perfume spray heaven classic
27551                                             rosemary
27552                          periperi sweet potato chips
27553                              green tea pure original
27554                       united dreams go far deodorant
Name: product_parsed, Length: 27087, dtype: object

In [30]:
DF.drop(['product','description'], axis=1, inplace = True)

In [31]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27087 entries, 0 to 27554
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   category            27087 non-null  float64
 1   sub_category        27087 non-null  object 
 2   brand               27087 non-null  object 
 3   sale_price          27087 non-null  float64
 4   market_price        27087 non-null  float64
 5   type                27087 non-null  object 
 6   rating              27087 non-null  float64
 7   description_parsed  27087 non-null  object 
 8   product_parsed      27087 non-null  object 
dtypes: float64(4), object(5)
memory usage: 2.1+ MB


In [32]:
DF.head()

,category,sub_category,brand,sale_price,market_price,type,rating,description_parsed,product_parsed
0,2.0,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,product contains garlic oil known help proper ...,garlic oil vegetarian capsule 500 mg
1,9.0,Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,product microwave safe without lid refrigerato...,water bottle orange
2,4.0,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,perfect gift occasions mother sister inlaws bo...,brass angle deep plain no2
3,4.0,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,multipurpose container attractive design made ...,cereal flip lid containerstorage jar assorted ...
4,2.0,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,nivea creme soft soap gives skin best care mus...,creme soft soap hands body


In [33]:
DF['sub_category'].nunique()

90

In [34]:
DF['sub_category'].value_counts()

Skin Care                   2217
Health & Medicine           1125
Storage & Accessories        998
Fragrances & Deos            990
Bath & Hand Wash             970
                            ... 
Mutton & Lamb                 20
Water                         17
Flower Bouquets, Bunches       8
Pork & Other Meats             5
Marinades                      1
Name: sub_category, Length: 90, dtype: int64

In [35]:
DF['type'].nunique()

426

In [36]:
DF['type'].value_counts()

Face Care                1456
Ayurveda                  535
Men's Deodorants          497
Shampoo & Conditioner     449
Containers Sets           411
                         ... 
Fresh & Frozen Pork         1
Roses                       1
Prawns & Shrimps            1
Specialty                   1
Turkey & Duck               1
Name: type, Length: 426, dtype: int64

In [37]:
DF['brand'].nunique()

2309

In [38]:
DF['brand'].value_counts()

Fresho              637
bb Royal            539
BB Home             427
DP                  243
Fresho Signature    170
                   ... 
Royal Touch           1
Amazon Series         1
Uproot                1
D'Acne                1
Baron De Bercy        1
Name: brand, Length: 2309, dtype: int64

def cumulatively_categorise(column,threshold=0.50,return_categories_list=True):
  #Find the threshold value using the percentage and number of instances in the column
  threshold_value=int(threshold*len(column))
  #Initialise an empty list for our new minimised categories
  categories_list=[]
  #Initialise a variable to calculate the sum of frequencies
  s=0
  #Create a counter dictionary of the form unique_value: frequency
  counts=Counter(column)

  #Loop through the category name and its corresponding frequency after sorting the categories by descending order of frequency
  for i,j in counts.most_common():
    #Add the frequency to the global sum
    s+=dict(counts)[i]
    #Append the category name to the list
    categories_list.append(i)
    #Check if the global sum has reached the threshold value, if so break the loop
    if s>=threshold_value:
      break
  #Append the category Other to the list
  categories_list.append('Other')

  #Replace all instances not in our new categories by Other  
  new_column=column.apply(lambda x: x if x in categories_list else 'Other')

  #Return transformed column and unique values if return_categories=True
  if(return_categories_list):
    return new_column,categories_list
  #Return only the transformed column if return_categories=False
  else:
    return new_column





#Call the function with a default threshold of 75%
transformed_brand,new_category_list=cumulatively_categorise(DF['brand'],return_categories_list=True)

In [39]:
X_train,X_test,y_train,y_test = train_test_split(DF[['category','sub_category','brand','market_price','rating','type']], 
                                                 DF['sale_price'], test_size =.3, random_state =111)

In [40]:
X_train.head()

,category,sub_category,brand,market_price,rating,type
24342,4.0,All Purpose Cleaners,Savlon,318.0,4.4,Disinfectant Spray & Cleaners
18072,2.0,Men's Grooming,Nova,1495.0,2.0,Moustache & Beard Care
8537,10.0,"Spreads, Sauces, Ketchup",Weikfield,55.0,4.2,Chilli & Soya Sauce
20102,10.0,Snacks & Namkeen,Dip,135.0,3.2,Namkeen & Savoury Snacks
15811,9.0,Kitchen Accessories,DP,139.0,5.0,Kitchen Tools & Other Accessories


In [41]:
y_train.head()

24342    256.76
18072    598.00
8537      49.00
20102    135.00
15811    139.00
Name: sale_price, dtype: float64

In [42]:
X_test.head()

,category,sub_category,brand,market_price,rating,type
3449,2.0,Health & Medicine,Herbal Hills,195.0,3.800000,Supplements & Proteins
22696,10.0,Chocolates & Candies,Kinder Joy,40.0,4.200000,Chocolates
10879,8.0,Oils & Vinegar,Basso,1599.0,5.000000,Extra Virgin Olive Oil
14914,2.0,Health & Medicine,Vedapure,1249.0,4.033333,Ayurveda
26147,9.0,Crockery & Cutlery,BP Bharat,979.0,4.300000,"Cups, Mugs & Tumblers"


In [43]:
y_test.head()

3449      195.0
22696      40.0
10879    1599.0
14914    1249.0
26147     789.0
Name: sale_price, dtype: float64

In [44]:
cat_vars = ['sub_category','brand','type']

In [45]:
encoder_dictionary ={}
for var in cat_vars:
    encoder_dictionary[var] = (X_train[var].value_counts()/len(X_train)).to_dict()

In [46]:
encoder_dictionary['brand']

{'Fresho': 0.02410337552742616,
 'bb Royal': 0.01977848101265823,
 'BB Home': 0.01550632911392405,
 'DP': 0.008808016877637131,
 'bb Combo': 0.006381856540084388,
 'Fresho Signature': 0.006170886075949367,
 'Amul': 0.005590717299578059,
 'Himalaya': 0.005379746835443038,
 'INATUR ': 0.0050105485232067514,
 'GoodDiet': 0.004852320675105486,
 'Dabur': 0.00479957805907173,
 'BIOTIQUE': 0.004483122362869198,
 'Colgate': 0.004483122362869198,
 'Nike': 0.004377637130801687,
 'Britannia': 0.004113924050632912,
 'Tasties': 0.004008438818565401,
 'Cello': 0.004008438818565401,
 'HappyChef': 0.003955696202531646,
 'Soulflower': 0.0039029535864978904,
 'Nakoda': 0.003850210970464135,
 'Organic Tattva': 0.0037447257383966246,
 'True Elements': 0.0037447257383966246,
 'Iveo ': 0.003639240506329114,
 'Dettol': 0.003639240506329114,
 'Graminway': 0.003586497890295359,
 'Aroma Magic ': 0.003586497890295359,
 'Loreal Paris': 0.003586497890295359,
 'StBotanica': 0.003586497890295359,
 'MTR': 0.003586497

In [47]:
for var in cat_vars:
    X_train[var] = X_train[var].map(encoder_dictionary[var])

In [48]:
X_train.head()

,category,sub_category,brand,market_price,rating,type
24342,4.0,0.017563,0.001477,318.0,4.4,0.002901
18072,2.0,0.030011,0.000949,1495.0,2.0,0.002901
8537,10.0,0.010443,0.001002,55.0,4.2,0.001108
20102,10.0,0.017932,0.000211,135.0,3.2,0.010232
15811,9.0,0.013027,0.008808,139.0,5.0,0.005116


In [49]:
X_train.isnull().sum()

category        0
sub_category    0
brand           0
market_price    0
rating          0
type            0
dtype: int64

In [50]:
for var in cat_vars:
    X_test[var] = X_test[var].map(encoder_dictionary[var])

In [51]:
X_test[X_test['brand'].isnull()]

,category,sub_category,brand,market_price,rating,type
1295,2.0,0.083439,NaN,75.0,4.300000,0.011814
11899,2.0,0.036445,NaN,399.0,4.100000,0.007278
9070,9.0,0.013555,NaN,375.0,3.766667,0.008808
25776,9.0,0.013555,NaN,290.0,3.933333,0.002321
25207,10.0,0.017932,NaN,60.0,4.300000,0.010232
...,...,...,...,...,...,...
8391,8.0,0.025949,NaN,275.0,4.766667,0.005380
11162,8.0,0.022205,NaN,260.0,5.000000,0.002321
815,2.0,0.041930,NaN,1195.0,3.000000,0.004378
2866,6.0,0.031962,NaN,55.0,2.000000,0.003692


In [52]:
X_test['brand'] = X_test['brand'].fillna(0)
X_test['type'] = X_test['type'].fillna(0)

In [53]:
X_test.isnull().sum()

category        0
sub_category    0
brand           0
market_price    0
rating          0
type            0
dtype: int64

In [54]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [59]:
print(model.score(X_train, y_train))

0.9319809980645293


In [55]:
print(model.score(X_test, y_test))

0.9352674148798632


In [57]:
y_pred = model.predict(X_test)

In [58]:
from sklearn.metrics import mean_absolute_error,mean_squared_error
  
mae = mean_absolute_error(y_true=y_test,y_pred=y_pred)
#squared True returns MSE value, False returns RMSE value.
mse = mean_squared_error(y_true=y_test,y_pred=y_pred) #default=True
rmse = mean_squared_error(y_true=y_test,y_pred=y_pred,squared=False)
  
print("MAE:",mae)
print("MSE:",mse)
print("RMSE:",rmse)

MAE: 51.90501004448051
MSE: 13695.56082109132
RMSE: 117.02803433832135


Frequency encoding resource - https://www.youtube.com/watch?v=o4UDAFd0DgY